In [ ]:
%load_ext autoreload
%autoreload 2

# Destination types with LDA

In [ ]:
!ls ../../../../data/wikivoyage/raw/

In [ ]:
data_dir = '../../../../data/wikivoyage/'

path_wiki_in  = data_dir + 'raw/enwikivoyage-20191001-pages-articles.xml.bz2'
# path_wiki_out = data_dir + 'clean/wikivoyage_metadata_all.csv'

In [ ]:
# import re
# import pandas as pd
import gensim 
from itertools import islice

## Load texts

In [ ]:
from stairway.wikivoyage.parsing import WikiCorpus

### apply scope

only do it for places in scope, or use the full wiki corpus to create the topics then apply it to places in scope?

### in case you want to see full texts:

In [ ]:
# wiki_new = wikivoyage.WikiCorpus(path_wiki_in, article_min_tokens=10)
# wiki_new.metadata = True

In [ ]:
# all_texts = []
# for (pageid, title, nr_tokens, patterns, text) in wiki_new.get_texts():
#     all_texts.append(text)

In [ ]:
# all_texts[1130]

### Parse text into tokens

To just parse the text and not return metadata set `metadata` to False (the default option):

In [ ]:
%%time
wiki_new = WikiCorpus(path_wiki_in, article_min_tokens=10)
wiki_new.metadata = False

In [ ]:
# example_tokens = []

# for tokens in islice(wiki_new.get_texts(), 2):
#     example_tokens.append(tokens)
    
# example_tokens[0]

In [ ]:
corpus = wiki_new.get_texts()

### Cleaning

In [ ]:
from gensim.parsing.preprocessing import STOPWORDS

# def tokenize(text):
#     return [token for token in simple_preprocess(text) if token not in STOPWORDS]


# LDA 


Helpful example notebook: https://radimrehurek.com/topic_modeling_tutorial/2%20-%20Topic%20Modeling.html

Gensim LDA needs a mmm object and a dictionary with words


In [ ]:
corpus = islice(wiki_new.get_texts(), 3)

In [ ]:
# access vectors directly using the standard corpus interface!!!!!!!!
for i in islice(wiki_new, 1):
    print(i)

In [ ]:
# access the dictionary with:
default_dict = wiki_new.dictionary

In [ ]:
corpus

In [ ]:
len(wiki_new.get_texts())

Create a [dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html).

Note creating the dictionary on the full wiki dump takes 17min. It contains 354k words.

In [ ]:
from gensim.corpora.dictionary import Dictionary

# Create a corpus from a list of texts
# common_dictionary = Dictionary(corpus)
# common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]

In [ ]:
%%time
dictionary = Dictionary(wiki_new.get_texts())


In [ ]:
len(dictionary)

In [ ]:
print(dictionary)

In [ ]:
# common_dictionary.doc2idx(['adding', 'after'])

In [ ]:
# common_dictionary.token2id

The dictionary object now contains all words that appeared in the corpus, along with how many times they appeared. Let's filter out both very infrequent words and very frequent words (stopwords), to clear up resources as well as remove noise:



In [ ]:
# calculate how many documents are 0.1% and use that as the `no_below` threshold?
len(wiki_new.get_texts())

In [ ]:
from copy import deepcopy

dictionary2 = deepcopy(dictionary)
# dictionary2.filter_extremes(no_below=100, no_above=0.6)
dictionary2.filter_extremes(keep_n=10)


In [ ]:
print(dictionary), print(dictionary2)

In [ ]:
default_dict.filter_extremes(no_below=10, no_above=0.6)

In [ ]:
print(wiki_new.dictionary)

In [ ]:
print(default_dict)

Clean the dictionary

In [ ]:
len(STOPWORDS)

In [ ]:
[common_dictionary.token2id[token] for token in STOPWORDS if token in ]

In [ ]:
common_dictionary.filter_tokens(bad_ids=[dct.token2id['ema']])

In [ ]:
dictionary2.token2id

In [ ]:
print(common_dictionary)

Make a matrix mapping, or convert documents into bag of words:

A streamed corpus and a dictionary is all we need to create bag-of-words vectors:

`doc2bow`: Convert document into the bag-of-words (BoW) format = list of (token_id, token_count) tuples.

In [ ]:
corpus

In [ ]:
common_corpus = [dictionary2.doc2bow(text) for text in islice(wiki_new.get_texts(), 1)]


In [ ]:
common_corpus

Let's store all those bag-of-words vectors into a file, so we don't have to parse the bzipped Wikipedia XML every time over and over:



In [ ]:
%time gensim.corpora.MmCorpus.serialize('./data/wiki_bow.mm', wiki_corpus)


In [ ]:
mm_corpus = gensim.corpora.MmCorpus('./data/wiki_bow.mm')
print(mm_corpus)

### LDA Modelling

Run the model.

[LDA docs](https://radimrehurek.com/gensim/models/ldamodel.html)

About the arguments:
- `Corpus` can either be: 
    - Stream of document vectors or, 
    - sparse matrix of shape (num_terms, num_documents).
- `id2word` ({dict of (int, str), gensim.corpora.dictionary.Dictionary}) – Mapping from word IDs to words. It is used to determine the vocabulary size, as well as for debugging and topic printing.



In [ ]:
from gensim.models.ldamodel import LdaModel

In [ ]:
# Train the model on the corpus.
lda = LdaModel(common_corpus, id2word=common_dictionary, num_topics=2, random_state=123)

In [ ]:
%%time 
# extract 100 LDA topics, using 1 pass and updating once every 1 chunk (10,000 documents)
# lda = LdaModel(corpus=mm, id2word=id2word, num_topics=100, update_every=1, passes=1)
lda = LdaModel(corpus=wiki_new, id2word=dictionary, num_topics=5, update_every=1, passes=1)

To run batch LDA (not online), train LdaModel with:

In [ ]:
# extract 100 LDA topics, using 20 full passes, no online updates
# lda = LdaModel(corpus=mm, id2word=id2word, num_topics=100, update_every=0, passes=20)

### Interpret results.

What would I like to find? Topics like:
- city: urban
- beach: Seaside
- cultural: historic, town, ancient, civilization, festivals, history, Remote monasteries, Historical sites:
- adventure: hiking, mountain biking, rock climbing, diving, white water rafting and so much more
- mountain -> skiing/ snowboarding or also hiking etc..? how different from nature?
- nature? -> what type of nature? safari? wildlife? mountainious, desert, glacial?
- countryside?
- town / village

In the old Stairway, we had:
- outdoors
- beach
- culture
- city

Triporati:
- city vs. outdoors -> Nature vs. water -> playing vs. relaxing
- culture vs local history -> adult vs. family

oude stairway: 
- nature	
- city	
- active	
- culture	
- relax


In [ ]:
print(lda)

In [ ]:
lda.print_topics()

In [ ]:
lda.show_topic(0, )

In [ ]:
# lda.get_topics()

In [ ]:
# lda.get_topic_terms(topicid, topn=10)

In [ ]:
# select top 10 words for each of the LDA topics
top_words = [[word for word, prob in lda.show_topic(topicno, topn=10)] for topicno in range(lda.num_topics)]
print(top_words)

### SAve / load model to disk

In [ ]:
from gensim.test.utils import datapath
>>>
>>> # Save model to disk.
>>> temp_file = datapath("model")
>>> lda.save(temp_file)
>>>
>>> # Load a potentially pretrained model from disk.
>>> lda = LdaModel.load(temp_file)

In [ ]:
>>> import logging
>>> import gensim
>>>
>>> logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
>>>
>>> # load id->word mapping (the dictionary), one of the results of step 2 above
>>> id2word = gensim.corpora.Dictionary.load_from_text('wiki_en_wordids.txt')
>>> # load corpus iterator
>>> mm = gensim.corpora.MmCorpus('wiki_en_tfidf.mm')
>>> # mm = gensim.corpora.MmCorpus('wiki_en_tfidf.mm.bz2') # use this if you compressed the TFIDF output
>>>
>>> print(mm)
MmCorpus(3931787 documents, 100000 features, 756379027 non-zero entries)